In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras_tuner import RandomSearch

# Generate synthetic stock data (replace with your dataset)
np.random.seed(42)
num_samples = 1000
closing_prices = np.sin(np.linspace(0, 20, num_samples)) + np.random.normal(0, 0.1, num_samples)

# Prepare the dataset
dates = pd.date_range(start="2020-01-01", periods=num_samples, freq="D")
data = pd.DataFrame({"ds": dates, "y": closing_prices})

# Normalize the closing prices
scaler = MinMaxScaler()
data['y'] = scaler.fit_transform(data['y'].values.reshape(-1, 1))

# Create sequences for 1D CNN
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 20
X, y = create_sequences(data['y'].values, sequence_length)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Reshape for 1D CNN input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define a function to build the model for hyperparameter tuning
def build_model(hp):
    model = Sequential()
    model.add(Conv1D(filters=hp.Int('filters', min_value=16, max_value=128, step=16), 
                     kernel_size=hp.Choice('kernel_size', values=[3, 5, 7]), 
                     activation='relu', 
                     input_shape=(sequence_length, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), 
                  loss='mse', 
                  metrics=['mse'])
    return model

# Use Keras Tuner for hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='mse',
    max_trials=10,
    executions_per_trial=2,
    directory='hyperparam_tuning',
    project_name='1d_cnn_stock_forecasting')

tuner.search(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: Filters: {best_hps.get('filters')}, Kernel Size: {best_hps.get('kernel_size')}, Units: {best_hps.get('units')}, Dropout: {best_hps.get('dropout')}, Learning Rate: {best_hps.get('learning_rate')}")

# Build and train the model with optimal hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Test Mean Squared Error: {mse}")

# Plot actual vs predicted
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_original = scaler.inverse_transform(y_pred)

plt.figure(figsize=(12, 6))
plt.plot(range(len(y_test_original)), y_test_original, label="True Prices")
plt.plot(range(len(y_pred_original)), y_pred_original, label="Predicted Prices")
plt.title("Stock Price Prediction using 1D CNN with Hyperparameter Tuning")
plt.legend()
plt.show()


C:\Users\Arindam\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Arindam\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\Arindam\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'keras_tuner'